# Explorative Data Analysis (EDA) – Online Retail Dataset

Dieses Notebook analysiert das **Online Retail Dataset** auf Basis einer performanten
Parquet-Datei (`retail_raw.parquet`).

Ziele dieses Notebooks:
- Verständnis der Datenstruktur und Datentypen
- Prüfung der Datenqualität (Missing Values, ungültige Werte, Duplikate)
- Ableitung erster betriebswirtschaftlicher Kennzahlen (KPIs)
- Vorbereitung für weiterführende Analysen (Customer Analytics, Time Series, ML)

Die Analyse erfolgt **realitätsnah**, d. h. inklusive typischer Probleme wie Retouren,
fehlenden Kundennummern und fehlerhaften Preisen.


## 1. Laden der Daten aus Parquet

In dieser Zelle wird das vorbereitete **Parquet-File** geladen.
Parquet ist ein spaltenorientiertes, performantes Speicherformat und eignet sich
besonders für analytische Workflows.

Ziel:
- Sicherstellen, dass die Daten korrekt geladen werden
- Erste visuelle Prüfung der Datensätze


In [ ]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("data/processed/retail_raw.parquet")

df = pd.read_parquet(DATA_PATH)
df.head()


## 2. Struktur- und Typübersicht

Hier wird ein erster struktureller Überblick über den Datensatz erzeugt:

- Anzahl der Zeilen und Spalten
- Datentypen jeder Spalte
- Statistische Basisinformationen (auch für kategoriale Spalten)

Dies ist ein zentraler Schritt, um:
- falsche Datentypen zu erkennen
- Auffälligkeiten früh zu identifizieren


In [ ]:
print("shape:", df.shape)
display(df.dtypes)
display(df.describe(include="all").T.head(20))


## 3. Analyse fehlender Werte

Fehlende Werte sind in realen Datensätzen normal und müssen bewusst behandelt werden.

In dieser Zelle wird:
- der Anteil fehlender Werte pro Spalte berechnet
- sichtbar gemacht, welche Variablen kritisch sind

Beispiel:
- Fehlende `CustomerID` → anonyme Käufe
- Fehlende Texte → mögliche Datenqualitätsprobleme


In [ ]:
missing = df.isna().mean().sort_values(ascending=False)
display(missing)


## 4. Datenqualitätsprüfungen (Retail-spezifisch)

Diese Prüfungen orientieren sich an typischen Problemen im Einzelhandel:

- Negative Mengen → Retouren oder Stornierungen
- Preise ≤ 0 → fehlerhafte Buchungen
- Fehlende CustomerID → Gastkäufe
- Duplikate → Mehrfachimporte oder Systemfehler

Die Ergebnisse helfen zu entscheiden, welche Daten
für Umsatzanalysen geeignet sind.


In [ ]:
checks = {
    "negative_quantity": (df["Quantity"] < 0).mean(),
    "zero_or_negative_unitprice": (df["UnitPrice"] <= 0).mean(),
    "missing_customerid": df["CustomerID"].isna().mean(),
    "duplicate_rows": df.duplicated().mean(),
}
pd.Series(checks).sort_values(ascending=False)


## 5. Umsatzberechnung und Filterung gültiger Verkäufe

Der Umsatz wird als:
> Quantity × UnitPrice

definiert.

Für betriebswirtschaftliche Analysen werden **nur echte Verkäufe** berücksichtigt:
- Menge > 0
- Preis > 0

Retouren und fehlerhafte Buchungen werden ausgeschlossen,
bleiben aber für spätere Analysen erhalten.


In [ ]:
df["Revenue"] = df["Quantity"] * df["UnitPrice"]

# "Sales" = nur echte Verkäufe (keine Returns, keine 0/neg prices)
sales = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)].copy()
sales["Revenue"] = sales["Quantity"] * sales["UnitPrice"]

sales[["InvoiceNo","InvoiceDate","Country","CustomerID","Revenue"]].head()


## 6. Zentrale Geschäftskennzahlen (KPIs)

In dieser Zelle werden grundlegende KPIs berechnet:

- Anzahl der Bestellungen
- Anzahl eindeutiger Kunden
- Anzahl belieferten Länder
- Gesamtumsatz
- Durchschnittlicher Bestellwert (Average Order Value)

Diese Kennzahlen bilden die Basis für Management-Reports
und strategische Entscheidungen.


In [ ]:
kpis = {
    "orders": sales["InvoiceNo"].nunique(),
    "customers": sales["CustomerID"].nunique(dropna=True),
    "countries": sales["Country"].nunique(),
    "total_revenue": float(sales["Revenue"].sum()),
    "avg_order_value": float(sales.groupby("InvoiceNo")["Revenue"].sum().mean()),
}
pd.Series(kpis)


## 7. Umsatzstärkste Länder und Produkte

Zum Abschluss der EDA werden:
- die umsatzstärksten Länder
- die umsatzstärksten Produkte

ermittelt.

Diese Analysen sind besonders relevant für:
- Marktpriorisierung
- Sortimentsentscheidungen
- Internationalisierungsstrategien


In [ ]:
top_countries = sales.groupby("Country")["Revenue"].sum().sort_values(ascending=False).head(10)
top_products = sales.groupby(["StockCode","Description"])["Revenue"].sum().sort_values(ascending=False).head(10)

display(top_countries)
display(top_products)
